In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('students_marks.csv')

# Ensure the dataset has exactly 50 rows (fill missing rows if necessary)
if len(df) < 50:
    missing_rows = 50 - len(df)
    df = pd.concat([df, pd.DataFrame([{}] * missing_rows)], ignore_index=True)

# Function to calculate internal marks
def calculate_internal_marks(row):
    # Fetch all test marks
    tests = [row['Class Test 1'], row['Class Test 2'], row['Class Test 3'], row['Class Test 4']]
    attendance = row['Attendance']
    grade = row['Overall Class Grade'] if pd.notnull(row['Overall Class Grade']) else 'C'  # Default grade to 'C' if missing

    # If attendance is 0, mark daily evaluation as NA and still calculate internal marks
    if attendance == 0:
        row['Class'] = 'NA'
    
    # Condition 1: If attendance < 75 (except 0 cases), it's debarred
    if attendance < 75 and attendance != 0:
        return "Debarred"

    # Process absent tests (0 marks) as per conditions
    adjusted_tests = []
    for test in tests:
        if test == 0:
            if attendance >= 85:
                # Replace based on Grade for >=85% attendance
                if grade == 'A':
                    adjusted_tests.append(18)
                elif grade == 'B':
                    adjusted_tests.append(17)
                else:  # Grade C
                    adjusted_tests.append(16)
            elif 75 <= attendance < 85:
                # Replace based on Grade for >=75% and <85% attendance
                if grade == 'A':
                    adjusted_tests.append(16)
                elif grade == 'B':
                    adjusted_tests.append(15)
                else:  # Grade C
                    adjusted_tests.append(14)
        else:
            adjusted_tests.append(test)

    # Select the best 3 tests after adjustment
    tests_sorted = sorted(adjusted_tests, reverse=True)
    best_3_tests = tests_sorted[:3]
    test_avg = sum(best_3_tests) / 3

    # Scale Class Test Marks to 40
    test_marks = (test_avg / 20) * 40  # Test component is out of 40

    # Attendance Marks Calculation (Max 5)
    if attendance >= 80:
        attendance_marks = 5
    elif attendance >= 75:
        attendance_marks = 3
    else:
        attendance_marks = 2

    # Class Performance Marks based on Grade (Max 5)
    grade_performance_marks = {'A': 5, 'B': 4, 'C': 3}
    performance_marks = grade_performance_marks.get(grade, 3)

    # Total Internal Marks out of 50
    internal_marks = test_marks + attendance_marks + performance_marks
    return round(internal_marks, 2)

# Apply the function to each student
df['Internal Marks (out of 50)'] = df.apply(calculate_internal_marks, axis=1)

# Export result to a new CSV file (optional)
df.to_csv('Final_Internal_Marks.csv', index=False)

# Print the output for review
print(df[['Name', 'Roll','Class Test 1','Class Test 2','Class Test 3','Class Test 4','Attendance','Overall Class Grade', 'Internal Marks (out of 50)']])


KeyError: "['Daily Evaluation'] not in index"